## Multi Agent-Systems


 A team is a group of agents that work together to achieve a common goal

In [2]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient,AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
from autogen_agentchat.messages import TextMessage
import os
from autogen_agentchat.agents import UserProxyAgent

from autogen_agentchat.conditions import TextMentionTermination,ExternalTermination

In [3]:


load_dotenv()
api_key = os.getenv('AZURE_OPENAI_API_KEY')
# model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)
model_client = AzureOpenAIChatCompletionClient(
    # azure_deployment="Basavaraj-Bijali",  # This is your deployment name, not the model name
    model="gpt-4o",  # Model name, e.g., "gpt-4"
    api_version="2024-12-01-preview",  # Use the API version your deployment supports
    azure_endpoint="https://tvscs-az-openai.openai.azure.com/",
    api_key=api_key
)

# Single Agent Approach

A single agent is going to create a short story for us

In [4]:
from autogen_agentchat.agents import (AssistantAgent)

story_agent = AssistantAgent(
    name = 'story_agent',
    model_client=model_client,
    system_message='Tou are a helpful AI assistant'
)

In [5]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 500 words',
        source='user'
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)

await test_simple_agent()

**The Dragon's Accord**  

In the kingdom of Eldoria, there was a legend that struck both fear and awe in the hearts of its people: the emerald-scaled dragon, Zephyrax, who guarded the ancient Forest of Whispers. For centuries, the forest had been off-limits to villagers who feared the dragon's wrath, but it also held the land's most fertile soil and rarest medicinal herbs. Many knights had tried to slay Zephyrax, but none returned.  

Sir Eryndor, a young but steadfast knight, was determined to succeed where others had failed. He bore no illusions of fame or fortune—his village was struggling, and drought and sickness were claiming lives. Zephyrax represented either salvation or ruin, and Sir Eryndor chose salvation. Armed with nothing but his sturdy blade and a heart of resolve, he ventured into the forbidden forest.  

The Forest of Whispers was breathtakingly beautiful, its air filled with the music of rustling leaves, faint birdsong, and the hints of ancient magic. Along the way, 

/Users/basavarajbijali/Desktop/Personal/Udemy_AI_AGENT_COURCE_Jun25/Autogen/autogen_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


# Multi Agent Team Approach

We will be using 3 agents to now write the story, each with its own work and mastery

RoundRobinGroupChat is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [6]:
primary_agnet = AssistantAgent(
    name = 'primary',
    model_client=model_client,
    system_message="You are a helffull AI assistant"
)

critic_agent = AssistantAgent(
    name = 'critic',
    model_client=model_client,
    system_message="Provide a constructive feedback. Respond with  'APROVE' when your feedbacks are addressed."
)

# ending_agent = AssistantAgent(
#     name = 'ending_writer',
#     model_client=model_client,
#     system_message="You write engaging endings. conclude the story with a twist."
# )

In [7]:
# from autogen_agentchat.conditions import TerminationCondition

# Create multiple termination conditions
approve_condition = TextMentionTermination('APPROVE')
done_condition = TextMentionTermination('DONE')
finished_condition = TextMentionTermination('FINISHED')

# # Combine them with OR logic
# def combined_termination(messages):
#     return (approve_condition(messages) or 
#             done_condition(messages) or 
#             finished_condition(messages))

termination_condition = approve_condition  or done_condition or finished_condition

In [17]:


from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [primary_agnet, critic_agent],
    termination_condition=termination_condition
)
# team

In [18]:

from autogen_agentchat.ui import Console
result = await Console(team.run_stream(task='Write a short poem about the fall season'))



---------- TextMessage (user) ----------
Write a short poem about the fall season
---------- TextMessage (primary) ----------
Golden leaves drift through the air,  
Whispers of autumn everywhere.  
Crisp winds hum a gentle tune,  
Beneath the glow of a harvest moon.  

Pumpkins perch on porch steps bright,  
Scarves pulled snug in amber light.  
The world slows down, a tranquil call,  
Nature’s grace—sweet, serene fall.  


/Users/basavarajbijali/Desktop/Personal/Udemy_AI_AGENT_COURCE_Jun25/Autogen/autogen_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- TextMessage (critic) ----------
Your poem beautifully captures the essence of the fall season with its vivid imagery and soothing rhythm. The use of descriptive language like "golden leaves," "crisp winds," and "amber light" paints an evocative picture of autumn and its charming serenity. 

One small suggestion for enhancement: consider adding a sensory detail that touches on the sound or smell of autumn—for example, the crunch of leaves underfoot or the aroma of spiced cider—to deepen the readers’ connection to the scene.

Other than that, it's lovely! Let me know if you make any adjustments.


---------- TextMessage (primary) ----------
Thank you for your thoughtful feedback! I appreciate your kind words and insightful suggestion. I’ve made a slight adjustment to the poem to include sensory details, as you recommended. Here’s the revised version:

---

Golden leaves drift through the air,  
Whispers of autumn everywhere.  
Crisp winds hum a gentle tune,  
With cider’s spic

In [25]:
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=75), metadata={}, content='Golden leaves drift through the air,  \nWhispers of autumn everywhere.  \nCrisp winds hum a gentle tune,  \nBeneath the glow of a harvest moon.  \n\nPumpkins perch on porch steps bright,  \nScarves pulled snug in amber light.  \nThe world slows down, a tranquil call,  \nNature’s grace—sweet, serene fall.  ', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=120, completion_tokens=115), metadata={}, content='Your poem beautifully captures the essence of the fall season with its vivid imagery and soothing rhythm. The use of descriptive language like "golden leaves," "crisp winds," and "amber light" paints an evocative picture of autumn and its charming serenity. \n\nOne small sugge

In [43]:
for messages in result:
    # print(messages)
    msg_list=messages[-1]
    # print(msg_list)
    for ms in msg_list:
        source=ms.source
        content=ms.content
        print('------',source,'------')
        print(content)
    break


------ user ------
Write a short poem about the fall season
------ primary ------
Golden leaves drift through the air,  
Whispers of autumn everywhere.  
Crisp winds hum a gentle tune,  
Beneath the glow of a harvest moon.  

Pumpkins perch on porch steps bright,  
Scarves pulled snug in amber light.  
The world slows down, a tranquil call,  
Nature’s grace—sweet, serene fall.  
------ critic ------
Your poem beautifully captures the essence of the fall season with its vivid imagery and soothing rhythm. The use of descriptive language like "golden leaves," "crisp winds," and "amber light" paints an evocative picture of autumn and its charming serenity. 

One small suggestion for enhancement: consider adding a sensory detail that touches on the sound or smell of autumn—for example, the crunch of leaves underfoot or the aroma of spiced cider—to deepen the readers’ connection to the scene.

Other than that, it's lovely! Let me know if you make any adjustments.


------ primary ------
Than

In [8]:

async def test_team():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 500 words',
        source='user'
    )

    result = await team.run(task=task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
        

await test_team()

/Users/basavarajbijali/Desktop/Personal/Udemy_AI_AGENT_COURCE_Jun25/Autogen/autogen_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


user : Write a short story about a brave knight and a dragon. Keep it up to 500 words
plot_writer : **The Last Flame**

Sir Corwin stood at the mouth of the Abyssal Crag, where the air burned with sulfur and grit. The erratic winds carried ash and the faint roar of a beast that had long haunted the realm: Solvyre, the Last Flame. The dragon’s fiery wrath had turned lush valleys to barren wastes and driven entire villages into exile. None who had faced it ever returned. Yet here Corwin stood, sword in hand, ready to challenge the creature.

He was no legendary hero, nor a knight born of nobility. Corwin had been just a farmer once—until Solvyre’s flames had claimed his village, taking with it the lives of his wife and son. Grief had forged his strength, and vengeance had sharpened his resolve. Now, he sought to face Solvyre, not as a savior of kings, but as a man who had lost everything.

At the heart of the cavern, Solvyre awaited. Its scales shimmered like molten gold beneath rivers o

# Comparision Table

## Single-Agent vs. Multi-Agent Comparison

| Aspect             | Single Agent                | Multi-Agent Team              |
|--------------------|-----------------------------|-------------------------------|
| **Creativity**     | Limited to one viewpoint    | Diverse ideas from each agent |
| **Depth**          | Basic story elements        | Detailed plot, characters, ending |
| **Flexibility**    | Stuck to one style          | Adaptable with specialized roles |
| **Setup Effort**   | Simple, one agent           | More agents, team setup       |

In [9]:
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short story about a brave knight and a dragon. Keep it up to 500 words', type='TextMessage'), TextMessage(source='plot_writer', models_usage=RequestUsage(prompt_tokens=44, completion_tokens=765), metadata={}, content="**The Knight and the Emberheart**\n\nIn the kingdom of Eldralis, where rolling green hills met jagged mountains, there lived tales of a dragon called Tharolyn, the Emberheart. Once a guardian of the realm, the dragon had descended into madness after the loss of its mate, unleashing fire and fury upon villages near its lair. For years, no champion had risen to face it—until Sir Kael, the youngest knight in the king's court, answered the call.\n\nKael was not notable for his size or his lineage, but his courage burned brighter than the castle fires. Armed with a sword forged from starmetal and a shield etched with protective runes, he journeyed to the foot of Emberpeak, a mounta

In [8]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

agents = [primary_agnet, critic_agent]

team = SelectorGroupChat(
    agents,
    model_client=model_client,  # For agent selection
    selector_prompt="Choose the best agent for this task",
    termination_condition=termination_condition
)

result = await Console(team.run_stream(task="Write a short story about a brave knight and a dragon. Keep it up to 500 words"))

/Users/basavarajbijali/Desktop/Personal/Udemy_AI_AGENT_COURCE_Jun25/Autogen/autogen_env/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_selector_group_chat.py:266: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(messages=select_speaker_messages)


---------- TextMessage (user) ----------
Write a short story about a brave knight and a dragon. Keep it up to 500 words
---------- TextMessage (primary) ----------
**The Knight and the Dragon**

Once upon a time in the rolling hills of Eldermere, a small village lived in constant fear of a dragon named Maelgor. The mighty beast with scales of molten gold dwelled atop Mount Virestone, and every month, it descended to demand offerings of livestock, gold, and jewels. If the offerings fell short, Maelgor unleashed fiery wrath upon the fields.

Desperate for peace, the village elders sent word to neighboring lands, pleading for a hero to rid them of the dragon. Among those who heeded the call was Sir Everin, a knight with a shining silver helm and a heart brimming with courage. He was neither the strongest nor the most skilled with a sword, but Everin was known far and wide for his unwavering resolve to do what was right.

When Everin arrived, he listened carefully to the villagers’ tales. 